In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

from scipy.stats import chi2_contingency
from scipy.stats import f_oneway
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score,confusion_matrix,recall_score,f1_score,roc_curve,roc_auc_score,accuracy_score
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDClassifier

from sklearn.linear_model import Lasso

In [2]:
tl_df=pd.read_csv("Telco_Customer_Churn_Data.csv",na_values=" ")
tl_df.dtypes
tl_df.isnull().sum()
tl_df.shape

(7043, 21)

In [3]:
## non sence columns
non_sence=[]
for col in tl_df:
    if len(tl_df[col].value_counts())==1 or len(tl_df[col].value_counts())==tl_df.shape[0]:
        non_sence.append(col)
non_sence

['customerID']

In [4]:
tl_df.drop(columns=["customerID"],inplace=True)

In [5]:
tl_df["TotalCharges"]=tl_df["TotalCharges"].fillna(tl_df["TotalCharges"].mean())

In [6]:
### correlation checking for  cat to cat chi sqr
corr_chi=[]
for col in tl_df:
    if tl_df[col].dtype=="object":
        new=pd.crosstab(tl_df[col],tl_df["Churn"])
        result=chi2_contingency(new)
        if result[1]<0.05:
            corr_chi.append(col)
corr_chi        


['Partner',
 'Dependents',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'Churn']

In [7]:
### correlation check for cont to cat
anova_corr=[]
for col in tl_df:
    if tl_df[col].dtype=="int64" or tl_df[col].dtype=="float64":
        new=list(tl_df["Churn"].value_counts().index)
        result=f_oneway(*(tl_df[tl_df["Churn"]==calls][col] for calls in new))
        if result[1]<0.05:
            anova_corr.append(col)
            
corr_df=corr_chi+anova_corr

In [8]:
corr_df=tl_df[corr_df]

In [9]:
corr_df.shape

(7043, 18)

In [10]:
y=corr_df["Churn"]
corr_df.drop(columns=["Churn"],inplace=True)

In [11]:
x_train,x_test,y_train,y_test=train_test_split(corr_df,y,test_size=0.20,random_state=43)
len(x_train),len(x_test),len(y_train),len(y_test)

(5634, 1409, 5634, 1409)

In [12]:
## scaling
scaler=StandardScaler()
cont=[i for i in x_train if x_train[i].dtype=="int64" or x_train[i].dtype=="float64"]
for col in cont:
    x_train[col]=scaler.fit_transform(np.array(x_train[col]).reshape(-1,1))
    x_test[col]=scaler.transform(np.array(x_test[col]).reshape(-1,1))

In [13]:
## encoding for cats
cat=[col for col in x_train if x_train[col].dtype=="object"]
oe_train=pd.get_dummies(x_train[cat])
oe_test=pd.get_dummies(x_test[cat])

In [14]:
train_oe,test_oe=oe_train.align(oe_test,join="inner",axis=1)

In [15]:
final_train=pd.concat([x_train[cont],train_oe],axis=1)
final_test=pd.concat([x_test[cont],test_oe],axis=1)

In [16]:
## fitting model
logreg=LogisticRegression()
logreg.fit(final_train,y_train)
test_pred=logreg.predict(final_test)
test_pred

array(['No', 'No', 'No', ..., 'No', 'Yes', 'No'], dtype=object)

In [17]:
## confusion matrix
confusion_matrix(test_pred,y_test)

array([[931, 160],
       [108, 210]], dtype=int64)

In [18]:
## test accuracy
accuracy_score(test_pred,y_test)

0.8097941802696949

In [19]:
## train accuracy
train_pred=logreg.predict(final_train)
accuracy_score(train_pred,y_train)

0.8061767838125665

In [20]:
recall_score(test_pred,y_test,pos_label="Yes")

0.660377358490566

In [21]:
precision_score(test_pred,y_test,pos_label="Yes")

0.5675675675675675

In [22]:
f1_score(test_pred,y_test,pos_label="Yes")

0.6104651162790697

In [23]:
pos_probs=logreg.predict_proba(final_test)[::,1]
pos_probs


array([0.31629872, 0.35808491, 0.14558155, ..., 0.12445471, 0.5560643 ,
       0.01624271])

In [24]:
fpr,tpr,threshold=roc_curve(y_test,pos_probs,pos_label="Yes")

In [25]:
plt.plot(fpr,tpr)

In [26]:
roc_auc_score(y_test,pos_probs)

0.8579975548214239

# applying sgd classifier model to overcome over fitting

In [27]:
las=Lasso()
sgd=SGDClassifier(penalty="l2",verbose=0.001)
sgd.fit(final_train,y_train)
sgd_test_pred=sgd.predict(final_test)
coeffs=sgd.coef_
len(coeffs[coeffs==0])

0

In [28]:
## test accuracy
accuracy_score(y_test,sgd_test_pred)

0.8019872249822569

In [29]:
## train accuracy
sgd_train_pred=sgd.predict(final_train)
accuracy_score(sgd_train_pred,y_train)

0.7900248491302805

In [30]:
## recall score
recall_score(sgd_train_pred,y_train,pos_label="Yes")

0.6007653061224489

In [31]:
###precision_score
precision_score(sgd_train_pred,y_train,pos_label="Yes")

0.628418945963976

In [32]:
## f1 score
f1_score(sgd_train_pred,y_train,pos_label="Yes")

0.6142810564069122

# DECISSION TREE CLASSIFIER

In [33]:
from sklearn.tree import DecisionTreeClassifier
dtc=DecisionTreeClassifier()
dtc.fit(final_train,y_train)
test_pred=dtc.predict(final_test)
## test accuracy
accuracy_score (test_pred,y_test)

0.7352732434350603

In [34]:
confusion_matrix(test_pred,y_test)

array([[832, 166],
       [207, 204]], dtype=int64)

In [35]:
recall_score(test_pred,y_test,pos_label="Yes")

0.49635036496350365

In [36]:
precision_score(test_pred,y_test,pos_label="Yes")

0.5513513513513514

In [37]:
f1_score(test_pred,y_test,pos_label="Yes")

0.5224071702944943

# ADABOOST ALGORITHM

In [38]:
from sklearn.ensemble import AdaBoostClassifier

In [39]:
adb=AdaBoostClassifier()
adb.fit(final_train,y_train)
adb_test_pred=adb.predict(final_test)

In [40]:
# test accuracy
accuracy_score(adb_test_pred,y_test)

0.8105039034776437

In [41]:
## train accuracy
accuracy_score(y_train,adb.predict(final_train))

0.8061767838125665

In [42]:
recall_score(adb_test_pred,y_test,pos_label="Yes")

0.6655948553054662

In [43]:
precision_score(adb_test_pred,y_test,pos_label="Yes")

0.5594594594594594

# gradient boost algorithm

In [44]:
from sklearn.ensemble import GradientBoostingClassifier

In [45]:
gbr=GradientBoostingClassifier()
gbr.fit(final_train,y_train)
accuracy_score(y_test,gbr.predict(final_test))

0.8197303051809794

In [46]:
recall_score(y_test,gbr.predict(final_test),pos_label="Yes")

0.5594594594594594

In [47]:
precision_score(y_test,gbr.predict(final_test),pos_label="Yes")

0.6946308724832215

# SUPPORT VECTOR MACHINE


In [48]:
from sklearn.svm import SVC

In [49]:
svc=SVC()
svc.fit(final_train,y_train)

SVC()

In [50]:
accuracy_score(y_test,svc.predict(final_test))

0.8090844570617459

In [51]:
recall_score(y_test,svc.predict(final_test),pos_label="Yes")

0.518918918918919

In [52]:
precision_score(y_test,svc.predict(final_test),pos_label="Yes")

0.6784452296819788

In [53]:
# HYPER PARAMETER TUNING
# Grid search cv

In [54]:
from sklearn.model_selection import GridSearchCV

In [55]:
parameters={"max_depth":[20,40,60,80,100],
           "min_samples_split":[2,4,6,8,10],
           "min_samples_leaf":[1,2,3,4,5],
           "max_leaf_nodes":[1,3,5,7]}

In [56]:
grd=GridSearchCV(gbr,param_grid=parameters,cv=6,verbose=20)

In [57]:
grd.fit(final_train,y_train)

Fitting 6 folds for each of 500 candidates, totalling 3000 fits
[CV 1/6; 1/500] START max_depth=20, max_leaf_nodes=1, min_samples_leaf=1, min_samples_split=2
[CV 1/6; 1/500] END max_depth=20, max_leaf_nodes=1, min_samples_leaf=1, min_samples_split=2;, score=nan total time=   0.0s
[CV 2/6; 1/500] START max_depth=20, max_leaf_nodes=1, min_samples_leaf=1, min_samples_split=2
[CV 2/6; 1/500] END max_depth=20, max_leaf_nodes=1, min_samples_leaf=1, min_samples_split=2;, score=nan total time=   0.0s
[CV 3/6; 1/500] START max_depth=20, max_leaf_nodes=1, min_samples_leaf=1, min_samples_split=2
[CV 3/6; 1/500] END max_depth=20, max_leaf_nodes=1, min_samples_leaf=1, min_samples_split=2;, score=nan total time=   0.0s
[CV 4/6; 1/500] START max_depth=20, max_leaf_nodes=1, min_samples_leaf=1, min_samples_split=2
[CV 4/6; 1/500] END max_depth=20, max_leaf_nodes=1, min_samples_leaf=1, min_samples_split=2;, score=nan total time=   0.0s
[CV 5/6; 1/500] START max_depth=20, max_leaf_nodes=1, min_samples_le

[CV 3/6; 7/500] END max_depth=20, max_leaf_nodes=1, min_samples_leaf=2, min_samples_split=4;, score=nan total time=   0.0s
[CV 4/6; 7/500] START max_depth=20, max_leaf_nodes=1, min_samples_leaf=2, min_samples_split=4
[CV 4/6; 7/500] END max_depth=20, max_leaf_nodes=1, min_samples_leaf=2, min_samples_split=4;, score=nan total time=   0.0s
[CV 5/6; 7/500] START max_depth=20, max_leaf_nodes=1, min_samples_leaf=2, min_samples_split=4
[CV 5/6; 7/500] END max_depth=20, max_leaf_nodes=1, min_samples_leaf=2, min_samples_split=4;, score=nan total time=   0.0s
[CV 6/6; 7/500] START max_depth=20, max_leaf_nodes=1, min_samples_leaf=2, min_samples_split=4
[CV 6/6; 7/500] END max_depth=20, max_leaf_nodes=1, min_samples_leaf=2, min_samples_split=4;, score=nan total time=   0.0s
[CV 1/6; 8/500] START max_depth=20, max_leaf_nodes=1, min_samples_leaf=2, min_samples_split=6
[CV 1/6; 8/500] END max_depth=20, max_leaf_nodes=1, min_samples_leaf=2, min_samples_split=6;, score=nan total time=   0.0s
[CV 2/6; 

[CV 4/6; 14/500] END max_depth=20, max_leaf_nodes=1, min_samples_leaf=3, min_samples_split=8;, score=nan total time=   0.0s
[CV 5/6; 14/500] START max_depth=20, max_leaf_nodes=1, min_samples_leaf=3, min_samples_split=8
[CV 5/6; 14/500] END max_depth=20, max_leaf_nodes=1, min_samples_leaf=3, min_samples_split=8;, score=nan total time=   0.0s
[CV 6/6; 14/500] START max_depth=20, max_leaf_nodes=1, min_samples_leaf=3, min_samples_split=8
[CV 6/6; 14/500] END max_depth=20, max_leaf_nodes=1, min_samples_leaf=3, min_samples_split=8;, score=nan total time=   0.0s
[CV 1/6; 15/500] START max_depth=20, max_leaf_nodes=1, min_samples_leaf=3, min_samples_split=10
[CV 1/6; 15/500] END max_depth=20, max_leaf_nodes=1, min_samples_leaf=3, min_samples_split=10;, score=nan total time=   0.0s
[CV 2/6; 15/500] START max_depth=20, max_leaf_nodes=1, min_samples_leaf=3, min_samples_split=10
[CV 2/6; 15/500] END max_depth=20, max_leaf_nodes=1, min_samples_leaf=3, min_samples_split=10;, score=nan total time=   0

[CV 5/6; 21/500] END max_depth=20, max_leaf_nodes=1, min_samples_leaf=5, min_samples_split=2;, score=nan total time=   0.0s
[CV 6/6; 21/500] START max_depth=20, max_leaf_nodes=1, min_samples_leaf=5, min_samples_split=2
[CV 6/6; 21/500] END max_depth=20, max_leaf_nodes=1, min_samples_leaf=5, min_samples_split=2;, score=nan total time=   0.0s
[CV 1/6; 22/500] START max_depth=20, max_leaf_nodes=1, min_samples_leaf=5, min_samples_split=4
[CV 1/6; 22/500] END max_depth=20, max_leaf_nodes=1, min_samples_leaf=5, min_samples_split=4;, score=nan total time=   0.0s
[CV 2/6; 22/500] START max_depth=20, max_leaf_nodes=1, min_samples_leaf=5, min_samples_split=4
[CV 2/6; 22/500] END max_depth=20, max_leaf_nodes=1, min_samples_leaf=5, min_samples_split=4;, score=nan total time=   0.0s
[CV 3/6; 22/500] START max_depth=20, max_leaf_nodes=1, min_samples_leaf=5, min_samples_split=4
[CV 3/6; 22/500] END max_depth=20, max_leaf_nodes=1, min_samples_leaf=5, min_samples_split=4;, score=nan total time=   0.0s


[CV 1/6; 28/500] END max_depth=20, max_leaf_nodes=3, min_samples_leaf=1, min_samples_split=6;, score=0.802 total time=   0.7s
[CV 2/6; 28/500] START max_depth=20, max_leaf_nodes=3, min_samples_leaf=1, min_samples_split=6
[CV 2/6; 28/500] END max_depth=20, max_leaf_nodes=3, min_samples_leaf=1, min_samples_split=6;, score=0.809 total time=   0.6s
[CV 3/6; 28/500] START max_depth=20, max_leaf_nodes=3, min_samples_leaf=1, min_samples_split=6
[CV 3/6; 28/500] END max_depth=20, max_leaf_nodes=3, min_samples_leaf=1, min_samples_split=6;, score=0.804 total time=   0.6s
[CV 4/6; 28/500] START max_depth=20, max_leaf_nodes=3, min_samples_leaf=1, min_samples_split=6
[CV 4/6; 28/500] END max_depth=20, max_leaf_nodes=3, min_samples_leaf=1, min_samples_split=6;, score=0.785 total time=   0.6s
[CV 5/6; 28/500] START max_depth=20, max_leaf_nodes=3, min_samples_leaf=1, min_samples_split=6
[CV 5/6; 28/500] END max_depth=20, max_leaf_nodes=3, min_samples_leaf=1, min_samples_split=6;, score=0.802 total tim

...

[CV 2/6; 479/500] END max_depth=100, max_leaf_nodes=7, min_samples_leaf=1, min_samples_split=8;, score=0.801 total time=   0.8s
[CV 3/6; 479/500] START max_depth=100, max_leaf_nodes=7, min_samples_leaf=1, min_samples_split=8
[CV 3/6; 479/500] END max_depth=100, max_leaf_nodes=7, min_samples_leaf=1, min_samples_split=8;, score=0.787 total time=   0.8s
[CV 4/6; 479/500] START max_depth=100, max_leaf_nodes=7, min_samples_leaf=1, min_samples_split=8
[CV 4/6; 479/500] END max_depth=100, max_leaf_nodes=7, min_samples_leaf=1, min_samples_split=8;, score=0.785 total time=   1.0s
[CV 5/6; 479/500] START max_depth=100, max_leaf_nodes=7, min_samples_leaf=1, min_samples_split=8
[CV 5/6; 479/500] END max_depth=100, max_leaf_nodes=7, min_samples_leaf=1, min_samples_split=8;, score=0.807 total time=   0.9s
[CV 6/6; 479/500] START max_depth=100, max_leaf_nodes=7, min_samples_leaf=1, min_samples_split=8
[CV 6/6; 479/500] END max_depth=100, max_leaf_nodes=7, min_samples_leaf=1, min_samples_split=8;, sco

[CV 3/6; 485/500] END max_depth=100, max_leaf_nodes=7, min_samples_leaf=2, min_samples_split=10;, score=0.793 total time=   0.8s
[CV 4/6; 485/500] START max_depth=100, max_leaf_nodes=7, min_samples_leaf=2, min_samples_split=10
[CV 4/6; 485/500] END max_depth=100, max_leaf_nodes=7, min_samples_leaf=2, min_samples_split=10;, score=0.783 total time=   0.8s
[CV 5/6; 485/500] START max_depth=100, max_leaf_nodes=7, min_samples_leaf=2, min_samples_split=10
[CV 5/6; 485/500] END max_depth=100, max_leaf_nodes=7, min_samples_leaf=2, min_samples_split=10;, score=0.810 total time=   0.8s
[CV 6/6; 485/500] START max_depth=100, max_leaf_nodes=7, min_samples_leaf=2, min_samples_split=10
[CV 6/6; 485/500] END max_depth=100, max_leaf_nodes=7, min_samples_leaf=2, min_samples_split=10;, score=0.799 total time=   0.8s
[CV 1/6; 486/500] START max_depth=100, max_leaf_nodes=7, min_samples_leaf=3, min_samples_split=2
[CV 1/6; 486/500] END max_depth=100, max_leaf_nodes=7, min_samples_leaf=3, min_samples_split=

[CV 4/6; 491/500] END max_depth=100, max_leaf_nodes=7, min_samples_leaf=4, min_samples_split=2;, score=0.786 total time=   0.8s
[CV 5/6; 491/500] START max_depth=100, max_leaf_nodes=7, min_samples_leaf=4, min_samples_split=2
[CV 5/6; 491/500] END max_depth=100, max_leaf_nodes=7, min_samples_leaf=4, min_samples_split=2;, score=0.806 total time=   0.7s
[CV 6/6; 491/500] START max_depth=100, max_leaf_nodes=7, min_samples_leaf=4, min_samples_split=2
[CV 6/6; 491/500] END max_depth=100, max_leaf_nodes=7, min_samples_leaf=4, min_samples_split=2;, score=0.799 total time=   0.8s
[CV 1/6; 492/500] START max_depth=100, max_leaf_nodes=7, min_samples_leaf=4, min_samples_split=4
[CV 1/6; 492/500] END max_depth=100, max_leaf_nodes=7, min_samples_leaf=4, min_samples_split=4;, score=0.796 total time=   0.9s
[CV 2/6; 492/500] START max_depth=100, max_leaf_nodes=7, min_samples_leaf=4, min_samples_split=4
[CV 2/6; 492/500] END max_depth=100, max_leaf_nodes=7, min_samples_leaf=4, min_samples_split=4;, sco

[CV 5/6; 497/500] END max_depth=100, max_leaf_nodes=7, min_samples_leaf=5, min_samples_split=4;, score=0.805 total time=   0.8s
[CV 6/6; 497/500] START max_depth=100, max_leaf_nodes=7, min_samples_leaf=5, min_samples_split=4
[CV 6/6; 497/500] END max_depth=100, max_leaf_nodes=7, min_samples_leaf=5, min_samples_split=4;, score=0.792 total time=   0.8s
[CV 1/6; 498/500] START max_depth=100, max_leaf_nodes=7, min_samples_leaf=5, min_samples_split=6
[CV 1/6; 498/500] END max_depth=100, max_leaf_nodes=7, min_samples_leaf=5, min_samples_split=6;, score=0.791 total time=   0.8s
[CV 2/6; 498/500] START max_depth=100, max_leaf_nodes=7, min_samples_leaf=5, min_samples_split=6
[CV 2/6; 498/500] END max_depth=100, max_leaf_nodes=7, min_samples_leaf=5, min_samples_split=6;, score=0.805 total time=   0.8s
[CV 3/6; 498/500] START max_depth=100, max_leaf_nodes=7, min_samples_leaf=5, min_samples_split=6
[CV 3/6; 498/500] END max_depth=100, max_leaf_nodes=7, min_samples_leaf=5, min_samples_split=6;, sco

GridSearchCV(cv=6, estimator=GradientBoostingClassifier(),
             param_grid={'max_depth': [20, 40, 60, 80, 100],
                         'max_leaf_nodes': [1, 3, 5, 7],
                         'min_samples_leaf': [1, 2, 3, 4, 5],
                         'min_samples_split': [2, 4, 6, 8, 10]},
             verbose=20)

In [58]:
grd_pred=grd.predict(final_test)

In [59]:
accuracy_score(y_test,grd_pred)

0.8197303051809794

In [60]:
grd.best_params_

{'max_depth': 20,
 'max_leaf_nodes': 3,
 'min_samples_leaf': 1,
 'min_samples_split': 2}

In [61]:
recall_score(y_test,grd_pred,pos_label="Yes")

0.5702702702702702

In [62]:
precision_score(y_test,grd_pred,pos_label="Yes")

0.6895424836601307

In [63]:
f1_score(y_test,grd_pred,pos_label="Yes")

0.6242603550295858

# HYPER PARAMETER TUNING